In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:

Number_level = 10;
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
Transition_matrix = load(fullname*".jld2","transition_matrix");


In [3]:
function reward(s, a,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
       if s==Number_level
            r=failure_penalty +  setup_cost/NumberUnits  + normal_operation  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
    if a==2
         if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits;
         return r
        else
            return 0.0
        end
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
        return r
    end
    end

reward (generic function with 3 methods)

In [4]:
T=zeros(Number_level,Number_level,3);


T[:,:,1].=Transition_matrix;

T[:,:,2].=Transition_matrix;

for j in 1:10
        T[j,:,3].=Transition_matrix[1,:];
        end

T[10,:,1].=Transition_matrix[1,:]
T[10,:,2].=Transition_matrix[1,:]

10-element view(::Array{Float64, 3}, 10, :, 2) with eltype Float64:
 0.2695279572108137
 0.16860741623354286
 0.1306613209439592
 0.09857168678543618
 0.09171796107138079
 0.09134806335671229
 0.09048875946531719
 0.04832973229444604
 0.00607725632113932
 0.004669846317252394

In [5]:

function generative(s, a, rng=23)       #s is state of a component and a is extend action of a component
        if s==Number_level
        s=1
        crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
       return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [6]:
function ValueIteration( γ)
    Q=zeros((Number_level,3))
    V= rand(Float64, (Number_level))*1000

    i=0
    p=[1 1 1 ]
    while i<=2000
#     while  i<=10000
    for s in 1:10
        Threads.@threads for a in 1:3
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:10 ))
    end

           p=[exp(lamb*(sum(Q[s,1]-findmin(Q[s,1:3])[1]))),exp(lamb*(Q[s,2]-findmin(Q[s,1:3])[1])),exp(lamb*(Q[s,3]-findmin(Q[s,1:3])[1]))]
            p=p./sum(p)
            V[s]=Q[s,1]*p[1]+Q[s,2]*p[2]+Q[s,3]*p[3]
        end
        i+=1
        end
    return Q,V
    end

ValueIteration (generic function with 1 method)

In [7]:




function decison(Q, s,n=0.2)
   a_final=[]
   if Number_level in s
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a], [2,3]))
            end
        return a_final
    end
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1]
    end
    a_final=repeat(1:1,NumberUnits);
    repair= argmax(i->s[i], 1:NumberUnits)
    a_final[repair]=3
    b=1
    for i in 1:NumberUnits
            if i!=repair
            a_final[i]=argmax(a->Q[s[i],a], [2,3])
            if a_final[i]==3
                b+=1
            end
            end
    end
        q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_final[i]]
    end
    if q0>q1
            return a_0
        else
            return a_final
        end
end

decison (generic function with 2 methods)

In [8]:
using DataFrames
df= DataFrame(Units=[],n=[],s=[],m=[],f=[],lambda=[],mean=[],std=[])

0×8 DataFrame

In [9]:

cost=[ 
 
    [0 -800 -200 -1000]
    ]

# print(Q)

for units in [20,30]
    for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in [100,10,5,1,0.5, 0.1,0.05,1e-2,0.5e-2,1e-3,0.5e-3,1e-4,0.5e-4,1e-5,0.5e-5,1e-6,0.5e-6,1e-7,0.5e-7,1e-8]
        global  lamb=lambda
    global NumberUnits=units
     Q,V=ValueIteration(.95)
         println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
    for i in 1:NumberUnits
         s[i],r[i]=generative(s[i],a[i],234);
          r2+=r[i]
            end 

    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
    results[h]=rk
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

20[0, -800, -200, -1000]
true
true
true
false


┌ Info:  started timer at: 2023-04-17T10:21:16.980
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


20
1.0
-34144.57569734362
2058.551849976428


┌ Info:           13.1955784s: 13 seconds, 195 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


false


...

┌ Info:           17.8001414s: 17 seconds, 800 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


false


┌ Info:  started timer at: 2023-04-17T10:29:48.121
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


30
1.0e-8
-47772.76618094414
2255.475820235613
-------------------------


┌ Info:           17.6493362s: 17 seconds, 649 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


In [10]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,30.0,0.0,-800.0,-200.0,-1000.0,1.0e-8,-47772.8,2255.48
2,30.0,0.0,-800.0,-200.0,-1000.0,5.0e-8,-47799.6,2255.07
3,30.0,0.0,-800.0,-200.0,-1000.0,1.0e-7,-47819.1,2269.92
4,30.0,0.0,-800.0,-200.0,-1000.0,5.0e-7,-47784.0,2288.12
5,30.0,0.0,-800.0,-200.0,-1000.0,1.0e-6,-47814.1,2281.59
6,30.0,0.0,-800.0,-200.0,-1000.0,5.0e-6,-47799.4,2264.6
7,30.0,0.0,-800.0,-200.0,-1000.0,1.0e-5,-47841.6,2266.88
8,30.0,0.0,-800.0,-200.0,-1000.0,5.0e-5,-47827.8,2268.15
9,30.0,0.0,-800.0,-200.0,-1000.0,0.0001,-47812.4,2316.33


In [11]:
using CSV
CSV.write("ACW-MDP.csv", df)

"ACW-MDP.csv"